---


In [ ]:

# Enter your information below
Student_Name = 'June Skeeter'
Student_Number = 123456789
print(f'GEOS 300 Assignment Submission for {Student_Name}: {Student_Number}')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime as dt

### Question 1 

Below is an example showing the first few rows of the BB climate summary data.  What values do you see listed for Incoming and outgoing $SW$ in the first row of the table, are these values reasonable?  Why or why not?  

* Make suer to specify the units in your answer. Check the metadata provided [here](https://github.com/GEOS300/AssignmentData/blob/main/Climate_Summary_BB_Metadata.md) to make sure you understand the variable names and units.


In [ ]:
# Import the data from github & parse the timestamp for each record
data_url='https://raw.githubusercontent.com/GEOS300/AssignmentData/main/Climate_Summary_BB.csv'
# Pandas (pd here) allows us to set a timestamp as an index which lets us easily parse time series data
df = pd.read_csv(data_url,parse_dates=['TIMESTAMP'],index_col=['TIMESTAMP'])
# Using this we can get a extra variables (DOY & HOUR) that will be helpful later
df['HOUR'] = df.index.hour
df['DOY'] = df.index.dayofyear

# Preview the table using .head()
df.head()

In [ ]:
# Insert your code or text answer here.  If writing a text answer in .ipynb, change the cell type to Markdown.  If writing a text answer in .Rmd, just delete the tick marks surrounding the block

### Question 2 

We're going to be looking at some data from the January 2024 cold-snap (Jan 12 to Jan 19) and snowstorm (Jan 17-Jan 19) during which record low temperatures were observed and over 25cm of snow was received across the Metro Vancouver area.  The table below shows some descriptive statistics for key variables during this time period.  What was the range of air temperatures observed over this time period?  How does it compare to the range of soil temperatures observed?


In [ ]:
# Set a time window and query the dataframe
Start = dt.strptime('2024-01-12 0000', '%Y-%m-%d %H%M')
End = dt.strptime('2024-01-20 0000', '%Y-%m-%d %H%M')

# Select a subset of the variables
Query_Cols = ['DOY','HOUR','SW_IN_1_1_1','LW_IN_1_1_1','SW_OUT_1_1_1', 'LW_OUT_1_1_1','TA_1_1_1','TS_1','RH_1_1_1']

# Run the query and save it to a new dataframe called "Selection"
Selection = df.loc[((df.index>=Start)&(df.index<End)),Query_Cols].copy()
                    
# Generate some descriptive statistics for the data
Selection.describe().round(3)

In [ ]:
# Insert your code or text answer here.  If writing a text answer in .ipynb, change the cell type to Markdown.  If writing a text answer in .Rmd, just delete the tick marks surrounding the block

### Question 3

The plot below shows traces of the four radiation component, air & soil temperature, and relative humidity are.  Looking at the plot, what stands out to you?  Comment on the progression of each radiative component over time, using the temperature and humidity data to support your discussion and help explain any patterns you might see.  Do you see any issues in the data?  If so, what is a possible explanation?


In [ ]:
# Create a plot with two sub-plots
# Plot the radiation components on the first sub-plot
# Plot Ta,Ts, and RH on the second sub-plot
fig,axes=plt.subplots(2,sharex=True)
plt.suptitle(f'Student #: {Student_Number}')

Selection.plot(ax=axes[0],
              y=['SW_IN_1_1_1','SW_OUT_1_1_1','LW_IN_1_1_1','LW_OUT_1_1_1'],
              color=['blue','red','blue','red'],
              style=['-','-','--','--'])
axes[0].legend(['SW In','SW Out','LW In','LW Out'],ncol=4)

# Note: You can use a markup language called LaTeX to format labels.  Replace Units,subscript, and superscript with the appropriate unit for radiative fluxes
axes[0].set_ylabel('Unit$_{subscript}^{superscript}$')
axes[0].set_ylim([0,475])
axes[0].set_title(f'Radiative Fluxes at CA-DBB Flux Station')
axes[0].grid()

# For this plot we're going to plot temperature on the left axes and RH on the right axes
axes[1].plot()
second_yaxes = axes[1].twinx()

# A simple "hack" to get the element from second_yaxes (RH_1_1_1) to show up with the legend items for the main yaxis
Selection['Dummy_Trace'] = np.nan
Selection.plot(ax=axes[1],
              y=['TA_1_1_1','TS_1','Dummy_Trace'],
              color=['red','red','blue'],
              style=['-','--',':'])
axes[1].legend(['Ta 2m','Ts 0.05m','RH 2m'],ncol=2,loc=4)
axes[1].set_ylabel('C$\degree$')
axes[1].set_ylim([-15,10])

Selection.plot(ax=second_yaxes,
              y=['RH_1_1_1'],
              color=['blue'],
              style=[':'],
              legend=False)
second_yaxes.set_ylabel('%')

axes[1].set_xlabel('Date')
axes[1].set_title('Temperature and Humidity')
axes[1].grid()

In [ ]:
# Insert your code or text answer here.  If writing a text answer in .ipynb, change the cell type to Markdown.  If writing a text answer in .Rmd, just delete the tick marks surrounding the block

### Question 4

The plot below shows mean daily albedo.  What explains sharp increase on the 17th?


In [ ]:
# Create a single plot bar showing mean daily albedo

# Calculate Albedo
Selection['Albedo'] = Selection['SW_OUT_1_1_1']/Selection['SW_IN_1_1_1']
# Filter out implausible values
Selection.loc[Selection['SW_OUT_1_1_1']>Selection['SW_IN_1_1_1'],'Albedo']=np.nan

# Resample to get mean daily Albedo, ignoring the NaN values
DailyAlbedo = Selection.resample('D').mean()
DailyAlbedo.index = DailyAlbedo.index.dayofyear

fig,ax=plt.subplots()
DailyAlbedo.plot.bar(ax=ax,y='Albedo',color='blue')
ax.set_ylabel('%')
ax.set_title('Daily Albedo')
plt.suptitle(f'Student #: {Student_Number}')

In [ ]:
# Insert your code or text answer here.  If writing a text answer in .ipynb, change the cell type to Markdown.  If writing a text answer in .Rmd, just delete the tick marks surrounding the block

## Question 5

The maximum half-hourly $SW_\downarrow$ observed during this time period occurred between 12:30 and 13:00 on January 13th.  Using the equations from lecture, calculate the Local Apparent Time (LAT), solar declination $\delta$, solar zenith angle $Z$, and extraterrestrial irradiance $I_ex$ for the center point of this observation period (12:45).  Given the value of $I_ex$, compared to observed $SW_\downarrow$, what is the approximate bulk transmissivity ($\Psi_a$) coefficient of the atmosphere over CA-DBB at this point in time?  Given the value of $I_ex$, compared to observed $SW_\downarrow$, what is the approximate bulk transmissivity ($\Psi_a$) coefficient of the atmosphere over CA-DBB at this point in time?  

**Note**: you can find the site's Lat/Lon and time zone info in the CA-DBB station metadata file.


In [ ]:
# Insert your code or text answer here.  If writing a text answer in .ipynb, change the cell type to Markdown.  If writing a text answer in .Rmd, just delete the tick marks surrounding the block

### Question 6

Now calculate $\Psi_a$ for 12:45 January 19; the observed value of $SW_\downarrow$ was between 12:30 and 13:00 was 58.375 W$^m{-2}$.  What explains the difference between these two days?


In [ ]:
# Insert your code or text answer here.  If writing a text answer in .ipynb, change the cell type to Markdown.  If writing a text answer in .Rmd, just delete the tick marks surrounding the block

### Question 7

Using the examples above as a template, calculate $R_n$ over the cold-snap.  Make a scatter plot showing $R_n$ on the y-axis and $SW_\downarrow$ on the x-axis.  Describe the relationship you.


In [ ]:
# Create a scatter plot

# fig,ax=plt.subplots()
# Selection.plot(ax=ax,kind='scatter',x='TA_1_1_1',y='TS_1')
# plt.suptitle(f'Student #: {Student_Number}')

In [ ]:
# Insert your code or text answer here.  If writing a text answer in .ipynb, change the cell type to Markdown.  If writing a text answer in .Rmd, just delete the tick marks surrounding the block

### Question 8

Using the examples above as a template, create a new selection to see how typical summertime conditions compare to severe winter time conditions.  Change the selection period so that it spans July 12th to July 19th, 2023.  Provide summary statistics of the same variables as those shown above and produce plots like the ones shown above as well. Discuss how each of these variables during this mid-summer period compare to those from the cold-snap period.


In [ ]:
# Insert your code or text answer here.  If writing a text answer in .ipynb, change the cell type to Markdown.  If writing a text answer in .Rmd, just delete the tick marks surrounding the block

In [ ]:
# Insert your code or text answer here.  If writing a text answer in .ipynb, change the cell type to Markdown.  If writing a text answer in .Rmd, just delete the tick marks surrounding the block

### Question 9

Find the timestamp of the observation interval when maximum $SW_\downarrow$ was observed for the mid-summer period.  You will use observations from this timestamp to answer each of the questions below.  **Note** our sites do not use daylight savings, so the UTC offset is fixed all year.  Use this information to calculate $I_ex$ and $\Psi_a$ for the mid-point of this observation period.  Discus how these two values compare between this observation, and the observation spanning between 12:30 and 13:00 on January 13th of 2024.  How does the magnitude differ and why does the relative timing of the maximum daily $SW_\downarrow$ differ?


In [ ]:
# Code Tip: getting the time and value when a variable reaches it's maximum

## Selection.loc[Selection['TA_1_1_1']==Selection['TA_1_1_1'].max(),'TA_1_1_1']

In [ ]:
# Insert your code or text answer here.  If writing a text answer in .ipynb, change the cell type to Markdown.  If writing a text answer in .Rmd, just delete the tick marks surrounding the block

### Question 10

Approximately 42% of the energy in the short-wave is within the PAR band (Photosynthetically active radiation that can be used by plants to perform photosynthesis). Based on this assumption, calculate the energy available to plants (also called: photosynthetic photon flux density, PPFD) at noon in $\mu \rm{mol}\, \rm{m}^{-2}\,\rm{s}^{-1}$. You can use the wavelength of yellow light ($\lambda 0 0.55 \mu m$) for your estimate.
**Hint**: Consult examples in reading package for radiation geometry.


In [ ]:
# Insert your code or text answer here.  If writing a text answer in .ipynb, change the cell type to Markdown.  If writing a text answer in .Rmd, just delete the tick marks surrounding the block

### Question 11

Let's assume the surface emissivity at CA-DBB is $\epsilon = 0.95$.  Using this assumption and the Stephan Boltzman law for grey bodies, estimate temperature of the land surface at CA-DBB for the time period identified above in question 9.


In [ ]:
# Insert your code or text answer here.  If writing a text answer in .ipynb, change the cell type to Markdown.  If writing a text answer in .Rmd, just delete the tick marks surrounding the block

### Question 12

Now use the same equation to estimate the "apparent" radiative sky temperature ($T_k$) from $LW_{\downarrow}$ for this timestamp.  "Apparent" means you should assume $\epsilon{a} = 1.0$. How would you interpret $T_k$?


In [ ]:
# Insert your code or text answer here.  If writing a text answer in .ipynb, change the cell type to Markdown.  If writing a text answer in .Rmd, just delete the tick marks surrounding the block

### Question 13

Using measured $LW_{\downarrow}$ and measured air temperature $T_a$, calculate the actual bulk emissivity of the atmosphere $\varepsilon_{a}$ at noon? [2]


In [ ]:
# Insert your code or text answer here.  If writing a text answer in .ipynb, change the cell type to Markdown.  If writing a text answer in .Rmd, just delete the tick marks surrounding the block

### Question 14

Using the Prata (1996) equation from lecture and measured $T_a$ and $RH$, what is the calculated apparent emissivity of the atmosphere and the calculated estimate of $LS_{\downarrow}$? How well do these approximations match with what we actually observed?


In [ ]:
# Insert your code or text answer here.  If writing a text answer in .ipynb, change the cell type to Markdown.  If writing a text answer in .Rmd, just delete the tick marks surrounding the block